In [16]:
import numpy as np
import time

In [2]:
import zhinst.toolkit as tk

In [3]:
mdc = tk.MultiDeviceController()
mdc.setup()

Successfully connected to data server at localhost8004 api version: 6


In [4]:
mdc.connect_hdawg("hd1", "dev8030", "1GbE")
mdc.connect_uhfqa("qa1", "dev2266", "1GbE")

Successfully connected to data server at localhost8004 api version: 6
Successfully connected to device DEV8030 on interface 1GBE
Added HDAWG: hd1
Successfully connected to data server at localhost8004 api version: 6
Successfully connected to device DEV2266 on interface 1GBE
Added UHFQA: qa1


In [5]:
mdc.hdawgs["hd1"].awgs[0].sequence_params

{'sequence_type': None,
 'sequence_parameters': {'target': 'hdawg',
  'clock_rate': 2400000000.0,
  'period': 0.0001,
  'trigger_mode': 'None',
  'repetitions': 1,
  'alignment': 'End with Trigger',
  'n_HW_loop': 1,
  'dead_time': 5e-06,
  'trigger_delay': 0,
  'latency': 1.6e-07,
  'trigger_cmd_1': '//\n',
  'trigger_cmd_2': '//\n',
  'wait_cycles': 0,
  'dead_cycles': 0}}

In [6]:
trigger_awg = mdc.hdawgs["hd1"].awgs[0]
slave_awgs = mdc.hdawgs["hd1"].awgs[1:]
readout = mdc.uhfqas["qa1"]

In [7]:
trigger_awg.set_sequence_params(
    sequence_type="Simple",
    repetitions=100,
    period=100e-6,
    trigger_mode="Send Trigger",
)
trigger_awg.queue_waveform([], [])
trigger_awg.compile_and_upload_waveforms()

current length of queue: 1
Compilation successful
hdawg0: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.016974210739135742 s


In [8]:
for awg in slave_awgs:
    awg.set_sequence_params(
        sequence_type="Simple",
        period=100e-6,
        repetitions=100,
        trigger_mode="External Trigger"
    )
    awg.queue_waveform(np.ones(800), -np.ones(800))
    awg.compile_and_upload_waveforms()

current length of queue: 1
Updated awg index to 1
Compilation successful
hdawg0: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.05200052261352539 s
current length of queue: 1
Updated awg index to 2
Compilation successful
hdawg0: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.012046575546264648 s
current length of queue: 1
Updated awg index to 3
Compilation successful
hdawg0: Sequencer status: ELF file uploaded
Upload of 1 waveforms took 0.013006925582885742 s


In [12]:
for ch in readout.channels[:3]:
    ch.enable()
    ch.readout_frequency = 100e6
    ch.readout_amplitude = 1.0
readout.awg.set_sequence_params(
    sequence_type="Readout",
    period=100e-6,
    repetitions=100,
    trigger_mode="External Trigger",
)
readout.awg.update_readout_params()
readout.awg.compile()

Compilation successful
uhfqa0: Sequencer status: ELF file uploaded


In [17]:
for awg in [trigger_awg]+slave_awgs+[readout.awg]:
    awg.output = "on"

In [19]:
for awg in slave_awgs + [readout.awg]:
    awg.run()
time.sleep(1)
trigger_awg.run()

hdawg0: Started AWG 1!
hdawg0: Started AWG 2!
hdawg0: Started AWG 3!
uhfqa0: Started AWG 0!
hdawg0: Started AWG 0!


In [20]:
for awg in slave_awgs + [readout.awg]:
    awg.stop()

hdawg0: Stopped AWG 1!
hdawg0: Stopped AWG 2!
hdawg0: Stopped AWG 3!
uhfqa0: Stopped AWG 0!
